In [109]:
import pandas as pd
import numpy as np
from pathlib import Path

path_to_data_raw = Path("../src/data/raw")
path_to_data_formatted = Path("../src/data/formatted")
path_to_data_processed = Path("../src/data/processed")

# Raw Data Cleanup
Fetch data from excel, fix errors and format as csv.

I don't think this should be included in the package.

In [104]:
ptq_raw = pd.read_excel(path_to_data_raw / "PTQ_year_month.xlsx").drop(["Year", "Month"], axis=1).rename({
    "Unnamed: 0": "date",
    "Precipitation (mm/day)": "p",
    "Discharge (m3/s)": "q",
    "Temperature 0C": "t"
}, axis=1)
ptq_raw["date"] = pd.to_datetime(ptq_raw["date"], dayfirst=True)
ptq_raw["q"] = ptq_raw["q"] * 86.4 / 1055 # Convert from m3/s to mm/day (Area of catchment is 1055km2)

et_raw = pd.read_excel(path_to_data_raw / "Daily_ET_1875_2007.xlsx")
et_raw.loc[9162, "Day"] = 28 # 31. Feb does not exist and 28. Feb is missing
et_raw["date"] = pd.to_datetime(et_raw[["Year", "Month", "Day"]])
et_raw = et_raw[~pd.isnull(et_raw["date"])].reset_index(drop=True).drop(["Month", "Year", "Day"], axis=1).rename({"ET (mm/day)": "epot"}, axis=1)

raw = ptq_raw.merge(et_raw, how="outer", on="date")
raw.to_csv(path_to_data_formatted / "timeseries_raw.csv", index=False)

In [105]:
pq_raw_reference = pd.read_excel(path_to_data_raw / "Reference_PQ_year_month.xlsx").drop(["Year", "Month"], axis=1).rename({
    "Unnamed: 0": "date",
    "Reference precipitation [mm/day]": "p",
    "Reference discharge (m3/s)": "q"
}, axis=1)
pq_raw_reference["date"] = pd.to_datetime(pq_raw_reference["date"], dayfirst=True)
pq_raw_reference["q"] = pq_raw_reference["q"] * 86.4 / 1055 # Convert from m3/s to mm/day (Area of catchment is 1055km2)
pq_raw_reference.to_csv(path_to_data_formatted / "timeseries_raw_reference.csv", index=False)

# Double Mass Correction
Observed precipitation and discharge may not be homogeneous over time (due to measurement error). The aim of this section is to correct for that.

In [106]:
from importlib import reload
from hydrosystem import double_mass_analysis
reload(double_mass_analysis)

<module 'hydrosystem.double_mass_analysis' from 'C:\\Users\\MUNRAS\\PycharmProjects\\NAM\\src\\hydrosystem\\double_mass_analysis\\__init__.py'>

In [107]:
target_columns = {"p", "q"}

raw = pd.read_csv(path_to_data_formatted / "timeseries_raw.csv")
reference = pd.read_csv(path_to_data_formatted / "timeseries_raw_reference.csv")
timeseries = raw.merge(reference, on="date", how="outer", suffixes=("_raw", "_reference"))
timeseries["year"] = pd.to_datetime(timeseries["date"]).dt.year
timeseries_annual = timeseries.drop(["date"], axis=1).groupby("year").agg("mean")

for k in target_columns:
    timeseries_annual[f"{k}_adjustment"] = double_mass_analysis.double_mass_analysis_with_changepoint(
        timeseries_annual[f"{k}_raw"],
        timeseries_annual[f"{k}_reference"]
    )
timeseries = timeseries.merge(timeseries_annual[[f"{k}_adjustment" for k in target_columns]], how="left", left_on="year", right_index=True)
for k in target_columns:
    timeseries[f"{k}_adjusted"] = timeseries[f"{k}_raw"] * timeseries[f"{k}_adjustment"]

timeseries_corrected = timeseries[["date", "p_adjusted", "t", "epot", "q_adjusted"]].rename({
    "p_adjusted": "p",
    "q_adjusted": "q"
}, axis=1)
timeseries_corrected.to_csv(path_to_data_processed / "timeseries_corrected.csv", index=False)

# Future Timeseries

In [111]:
# Read the delta change method values from the GEUS table
df_copilot = pd.read_csv(path_to_data_raw / "delta_change_copilot.csv")
df_copilot = df_copilot[np.logical_and.reduce((df_copilot["region"]=="DK5", df_copilot["scenario"]=="far", df_copilot["metric"]=="dc"))]
df_copilot.drop(["region", "scenario", "metric", "page"], axis=1, inplace=True)
df_copilot["month"] = np.select(
    [
        df_copilot["month"]=="Jan",
        df_copilot["month"]=="Feb",
        df_copilot["month"]=="Mar",
        df_copilot["month"]=="Apr",
        df_copilot["month"]=="May",
        df_copilot["month"]=="Jun",
        df_copilot["month"]=="Jul",
        df_copilot["month"]=="Aug",
        df_copilot["month"]=="Sep",
        df_copilot["month"]=="Oct",
        df_copilot["month"]=="Nov",
        df_copilot["month"]=="Dec",
    ],
    np.arange(1,13)
)

In [114]:
# Apply delta changes (factor for precipitation/epot and absolute for temperature)
timeseries_reference = pd.read_csv(path_to_data_processed / "timeseries_corrected.csv")
timeseries_reference["date"] = pd.to_datetime(timeseries_reference["date"])
timeseries_reference = timeseries_reference[np.logical_and(
    timeseries_reference["date"].dt.year >= 1971,
    timeseries_reference["date"].dt.year <= 2000
)]
timeseries_reference["month"] = timeseries_reference["date"].dt.month
timeseries_reference = timeseries_reference.iloc[:-1] # Drop the last day, because it needs to match future period length

precipitation_future = timeseries_reference[["date", "month", "p"]].copy()
precipitation_future["date_future"] = pd.date_range("2071-01-01", "2100-12-31")
precipitation_future = precipitation_future.merge(
    df_copilot[df_copilot["variable"]=="Precipitation"].drop(["variable"], axis=1),
    how="left",
    on="month"
)
precipitation_future["p_future"] = precipitation_future["p"] * precipitation_future["value"]

evapotranspiration_future = timeseries_reference[["date", "month", "epot"]].copy()
evapotranspiration_future["date_future"] = pd.date_range("2071-01-01", "2100-12-31")
evapotranspiration_future = evapotranspiration_future.merge(
    df_copilot[df_copilot["variable"]=="Reference ET"].drop(["variable"], axis=1),
    how="left",
    on="month"
)
evapotranspiration_future["epot_future"] = evapotranspiration_future["epot"] * evapotranspiration_future["value"]

temperature_future = timeseries_reference[["date", "month", "t"]].copy()
temperature_future["date_future"] = pd.date_range("2071-01-01", "2100-12-31")
temperature_future = temperature_future.merge(
    df_copilot[df_copilot["variable"]=="Temperature"].drop(["variable"], axis=1),
    how="left",
    on="month"
)
temperature_future["t_future"] = temperature_future["t"] + temperature_future["value"]

# Merge results, save to csv
future = precipitation_future[["date_future", "model", "p_future"]].merge(
    temperature_future[["date_future", "model", "t_future"]].merge(
        evapotranspiration_future[["date_future", "model", "epot_future"]],
        how="inner",
        on=["date_future", "model"]
    ),
    how="inner",
    on=["date_future", "model"]
).rename({
    "date_future": "date",
    "p_future": "p",
    "t_future": "t",
    "epot_future": "epot",
}, axis=1)
future.to_csv(path_to_data_processed / "timeseries_future_deltachange.csv", index=False)

In [116]:
def ingest_model(df, model):
    df = df.copy()
    df["model"] = model
    return df

precip_dbs = pd.concat([ingest_model(df,k) for k,df in pd.read_excel(path_to_data_raw / "Precip_DBS.xlsx", sheet_name=None).items()])
et_dbs = pd.concat([ingest_model(df,k) for k,df in pd.read_excel(path_to_data_raw / "RefET_DBS.xlsx", sheet_name=None).items()])
temp_dbs = pd.concat([ingest_model(df,k) for k,df in pd.read_excel(path_to_data_raw / "Temp_DBS.xlsx", sheet_name=None).items()])

dbs = precip_dbs.merge(
    et_dbs.merge(
        temp_dbs,
        on=["Time", "model"],
        how="inner"
    ),
    on=["Time", "model"],
    how="inner"
)
dbs.rename({
    "Time": "date",
    "Precip (mm)": "p",
    "RefET (mm)": "epot",
    "Temp (°C)": "t",
}, axis=1, inplace=True)
dbs.to_csv(path_to_data_processed / "timeseries_future_dbs.csv", index=False)